In [1]:
import numpy as np
import pandas as pd
import seaborn
import matplotlib.pyplot as plt
import datetime
import csv
import os
from sklearn.metrics import r2_score, mean_squared_error

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from modules.prediction import load_all_data
from modules.prediction import precrime_train_test_split
from modules.prediction import load_splits
from modules.prediction import create_all_splits
from modules.prediction import sample_model
from modules.poly_ridge import poly_ridge_model
from modules.fancy_time_series import fancy_time_series_model
from modules.eval_model import eval_predictions
from modules.prediction import create_test_period

In [3]:
crime_data = load_all_data()

/Users/jsamet/anaconda2/envs/py3k/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
split_future = create_test_period(datetime.date(2017,10,1),datetime.date(2018,9,30))

In [5]:
train_test_data = create_all_splits(crime_data, {'future': split_future})

In [6]:
X_train_future, _, y_train_future, _ = train_test_data['future']

In [7]:
average_weather = X_train_future.groupby([
    'COMPLAINT_MONTH',
    'COMPLAINT_DAY'
])[[
    'temperature',
    'precipIntensity'
]].mean().reset_index()

precinct_codes = pd.DataFrame(X_train_future['ADDR_PCT_CD'].unique(), columns=['ADDR_PCT_CD'])
precinct_codes['key'] = 1

gimme_a_datetime = split_future.copy()
gimme_a_datetime.columns = [c[5:].lower() for c in gimme_a_datetime.columns]
gimme_a_datetime['dayofweek'] = pd.to_datetime(gimme_a_datetime[['year', 'month', 'day']]).dt.dayofweek
gimme_a_datetime.columns = ['COMPLAINT_{0}'.format(c.upper()) for c in gimme_a_datetime.columns]
gimme_a_datetime['key'] = 1

dates_and_precincts = pd.merge(gimme_a_datetime, precinct_codes, how='outer')
dates_and_precincts.drop('key', axis=1, inplace=True)


In [8]:
X_test_future = pd.merge(dates_and_precincts, average_weather)
y_poly_future = poly_ridge_model(X_train_future, y_train_future, X_test_future)

In [9]:
crime_data_with_poly_future = pd.concat(
    [crime_data, pd.merge(X_test_future, y_poly_future)],
    ignore_index=True
)
train_test_data_with_poly_future = create_all_splits(crime_data_with_poly_future, {'ts_future': split_future})
X_train_ts_future, X_test_ts_future, y_train_ts_future, y_test_ts_future = train_test_data_with_poly_future['ts_future']
y_ts_future = fancy_time_series_model(X_train_ts_future, y_train_ts_future, X_test_ts_future, y_test_ts_future).reset_index().drop('index', axis=1)

In [22]:
y_hybrid_future = (y_poly_future + y_ts_future)/2
y_hybrid_future.to_csv('../precrime_data/future_predictions.csv', quoting=csv.QUOTE_NONNUMERIC)